In [3]:
from typing import Any
import json
import datetime
import re
import itertools
from cads_catalogue_api_service import constrictor
import inputs

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
form = constrictor.parse_form(inputs.form)
selection = constrictor.parse_selection(inputs.selection2)
combinations = constrictor.parse_combinations(inputs.combinations2)
constrictor.apply_constraints(form, selection, combinations)

{'date': ['1980-01-01/2011-12-31'],
 'level': ['1000', '500'],
 'year': [1980,
  1981,
  1982,
  1983,
  1984,
  1985,
  1986,
  1987,
  1988,
  1989,
  1990,
  1991,
  1992,
  1993,
  1994,
  1995,
  1996,
  1997,
  1998,
  1999,
  2000,
  2001,
  2002,
  2003,
  2004,
  2005,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011],
 'month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 'day': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31]}

In [8]:
constrictor.convert_ymd_to_dates_set({"2000", "2002"}, {"02"}, {"1", "29"})

{datetime.datetime(2000, 2, 1, 0, 0),
 datetime.datetime(2000, 2, 29, 0, 0),
 datetime.datetime(2002, 2, 1, 0, 0)}

In [73]:
def test_convert_date_range_to_date_set():
    start = "2000-01-01"
    stop = "2000-01-31"
    separator = ";"
        
    assert constrictor.convert_date_range_to_date_set(f"{start}{separator}{stop}") \
           == {
               datetime.datetime.strptime(start, "%Y-%m-%d")
               + datetime.timedelta(days=x) for x in range(31)}
test_convert_date_range_to_date_set()

In [71]:
def get_max_date(combinations):
    dates = []
    for combination in combinations:
        if "date" in combination.keys():
            for date in combination["date"]:
                dates.append(datetime.strptime(re.split(";|/", date)[0], '%Y-%m-%d'))
    return min(dates)

def get_min_date(combinations):
    dates = []
    for combination in combinations:
        if "date" in combination.keys():
            for date in combination["date"]:
                try:
                    dates.append(datetime.strptime(re.split(";|/", date)[1], '%Y-%m-%d'))
                except IndexError:
                    dates.append(datetime.strptime(re.split(";|/", date)[0], '%Y-%m-%d'))

    return max(dates)
    
def get_available_time_range(combinations):
    start = get_start_date(combinations)
    stop = get_stop_date(combinations)
    return f"{start.strftime('%Y-%m-%d')}/{stop.strftime('%Y-%m-%d')}"



In [10]:
y = {"2000","2001","2020"}
m = {"01","03","02"}
d = {"01", "02", "28", "29", "31"}
t = {"00:00", "03:00", "12:00", "21:00"}

def compute_ymdt_combinations(y,m,d,t):
    combinations = set()
    for x in set(itertools.product(y, m, d, t)):
        try:
            hh, mm = x[3].split(":")
            s = datetime(int(x[0]), int(x[1]), int(x[2]), int(hh), int(mm))
            combinations.add(s)
        except ValueError:
            continue
    return combinations



compute_ymd_combinations(y,m,d)

{datetime.datetime(2000, 1, 1, 0, 0),
 datetime.datetime(2000, 1, 2, 0, 0),
 datetime.datetime(2000, 1, 28, 0, 0),
 datetime.datetime(2000, 1, 29, 0, 0),
 datetime.datetime(2000, 1, 31, 0, 0),
 datetime.datetime(2000, 2, 1, 0, 0),
 datetime.datetime(2000, 2, 2, 0, 0),
 datetime.datetime(2000, 2, 28, 0, 0),
 datetime.datetime(2000, 2, 29, 0, 0),
 datetime.datetime(2000, 3, 1, 0, 0),
 datetime.datetime(2000, 3, 2, 0, 0),
 datetime.datetime(2000, 3, 28, 0, 0),
 datetime.datetime(2000, 3, 29, 0, 0),
 datetime.datetime(2000, 3, 31, 0, 0),
 datetime.datetime(2001, 1, 1, 0, 0),
 datetime.datetime(2001, 1, 2, 0, 0),
 datetime.datetime(2001, 1, 28, 0, 0),
 datetime.datetime(2001, 1, 29, 0, 0),
 datetime.datetime(2001, 1, 31, 0, 0),
 datetime.datetime(2001, 2, 1, 0, 0),
 datetime.datetime(2001, 2, 2, 0, 0),
 datetime.datetime(2001, 2, 28, 0, 0),
 datetime.datetime(2001, 3, 1, 0, 0),
 datetime.datetime(2001, 3, 2, 0, 0),
 datetime.datetime(2001, 3, 28, 0, 0),
 datetime.datetime(2001, 3, 29, 0, 0)

In [63]:
def convert_date_range_to_date_set(date_range):
    start_stop_interval = re.split(";|/", date_range)
    match len(start_stop_interval):
        case 1:
            return datetime.strptime(start_stop_interval[0], '%Y-%m-%d') 
        case 2:
            start = datetime.strptime(start_stop_interval[0], '%Y-%m-%d')
            stop = datetime.strptime(start_stop_interval[1], '%Y-%m-%d')
            return [start + timedelta(days=x) for x in range((stop-start).days + 1)]

convert_date_range_to_date_set("1990-01-01/2011-12-31")[:5]

    

[datetime.datetime(1990, 1, 1, 0, 0),
 datetime.datetime(1990, 1, 2, 0, 0),
 datetime.datetime(1990, 1, 3, 0, 0),
 datetime.datetime(1990, 1, 4, 0, 0),
 datetime.datetime(1990, 1, 5, 0, 0)]

In [ ]:
from datetimerange import DateTimeRange

def gen_time_range_from_string(string: str) -> DateTimeRange:
        dates = re.split(";|/", string)
        time_range = DateTimeRange(dates[0], dates[1])
        time_range.start_time_format = "%Y-%m-%d"
        time_range.end_time_format = "%Y-%m-%d"
        if time_range.is_valid_timerange():
            return time_range
        else:
            raise ValueError('Selected start date must be before end date')

def drop_unavailable(
    selected_time_range: set[str],
    valid_combinations: list[dict[str, set[Any]]],
) -> dict[str, set[Any]]:
    
    selected_time_range = gen_time_range_from_string(selected_time_range)
    for i in range(len(valid_combinations)-1, -1, -1):
        ok = False;
        valid_time_ranges = [gen_time_range_from_string(string) for string in valid_combinations[i]["date"]]
        for valid_time_range in valid_time_ranges:
            if selected_time_range in valid_time_range:
                ok = True
                break
        if not ok:
            del valid_combinations[i]
        else:
            valid_combinations[i].pop("date")
    return valid_combinations


def apply_constraints(valid_combinations, selection):
    if ("date" in valid_combinations[0]) & ("date" in selection.keys()):
        valid_combinations = drop_unavailable(next(iter(selection["date"])), valid_combinations)
        date = selection.pop("date") # pop out "date" otherwise get_possible_values will not work
        possible_values = get_possible_values(selection, valid_combinations)
        if possible_values:
            possible_values.setdefault("date", date) # get "date" back in, use selected time range
    else:
        possible_values = get_possible_values(selection, valid_combinations)
    return possible_values

In [ ]:
path = "/Users/nicola/devel/notebooks/data/constraints/eac4_constraints.json"
valid_combinations = load_combinations(path)

current_selection: dict[str, set[Any]] = {
        # valid time range: {'2003-01-01/2021-12-31'},
        #'date': {'2040-01-02/2080-01-13'},
        #'date': {'1980-01-02/1990-01-13'},
        #'date': {'2000-01-02/2010-01-13'},
        #'date': {'2010-01-02/2030-01-13'},
        'date': {'2005-01-02/2010-01-13'},
        'variable': {'10m_u_component_of_wind'},
        'time': {'00:00'},
}
       
path = "/Users/nicola/devel/notebooks/data/constraints/cmip6_constraints.json"
valid_combinations = load_combinations(path)

current_selection: dict[str, set[Any]] = {
    "experiment" : {"historical"},
    "variable": {"specific_humidity"},
    "year": {"2014", "2000"}
}

apply_constraints(valid_combinations, current_selection)

path = "/Users/nicola/devel/notebooks/data/constraints/gfas_constaints.json"
valid_combinations = load_combinations(path)

current_selection: dict[str, set[Any]] = {
    "variable": {"wildfire_overall_flux_of_burnt_carbon"},
    "date": {"2003-01-01/2022-10-09"}
}
apply_constraints(valid_combinations, current_selection)